In [ ]:
try:
    from openmdao.utils.notebook_utils import notebook_mode
except ImportError:
    !python -m pip install openmdao[notebooks]

# VectorMagnitudeComp

`VectorMagnitudeComp` computes the magnitude (L2 norm) of a single input of some given length.
It may be vectorized to provide the result at one or more points simultaneously.

$$
    \lvert a_i \rvert = \sqrt{\bar{a}_i \cdot \bar{a}_i}
$$

## VectorMagnitudeComp Options

The default `vec_size` is 1, providing the magnitude of $a$ at a singlepoint.  The length of $a$ is provided by option `length`.

Other options for VectorMagnitudeComp allow the user to rename the input variable $a$ and the output $a\_mag$, as well as specifying their units.

In [ ]:
import openmdao.api as om
om.show_options_table("openmdao.components.vector_magnitude_comp.VectorMagnitudeComp")

## VectorMagnitudeComp Constructor

The call signature for the `VectorMagnitudeComp` constructor is:

```{eval-rst}
    .. automethod:: openmdao.components.vector_magnitude_comp.VectorMagnitudeComp.__init__
        :noindex:
```

## VectorMagnitudeComp Usage

There are often situations when numerous magnitudes need to be computed, essentially in parallel.
You can reduce the number of components required by having one `VectorMagnitudeComp` perform multiple operations.
This is also convenient when the different operations have common inputs.

The `add_magnitude` method is used to create additional magnitude calculations after instantiation.

```{eval-rst}
    .. automethod:: openmdao.components.vector_magnitude_comp.VectorMagnitudeComp.add_magnitude
       :noindex:
```

## VectorMagnitudeComp Example

In the following example VectorMagnitudeComp is used to compute the radius vector magnitude
given a radius 3-vector at 100 points simultaneously. Note the use of `in_name` and `mag_name` to assign names to the inputs and outputs. Units are assigned using `units`.  The units of the output magnitude are the same as those for the input.

In [ ]:
import numpy as np
import openmdao.api as om

n = 100

p = om.Problem()

comp = om.VectorMagnitudeComp(vec_size=n, length=3,
                              in_name='r', mag_name='r_mag', units='km')

p.model.add_subsystem(name='vec_mag_comp', subsys=comp,
                      promotes_inputs=[('r', 'pos')])

p.setup()

p.set_val('pos', 1.0 + np.random.rand(n, 3))

p.run_model()

# Verify the results against numpy.dot in a for loop.
expected = []
for i in range(n):
    a_i = p.get_val('pos')[i, :]
    expected.append(np.sqrt(np.dot(a_i, a_i)))

    actual_i = p.get_val('vec_mag_comp.r_mag')[i]
    rel_error = np.abs(expected[i] - actual_i)/actual_i
    assert rel_error < 1e-9, f"Relative error: {rel_error}"

print(p.get_val('vec_mag_comp.r_mag'))

In [ ]:
from openmdao.utils.assert_utils import assert_near_equal

assert_near_equal(p.get_val('vec_mag_comp.r_mag'), np.array(expected))

## VectorMagnitudeComp Example with Multiple Magnitudes

Note that, when defining multiple magnitudes, an input name in one call to `add_magnitude` may not be an output name in another call, and vice-versa.

In [ ]:
import numpy as np

n = 100

p = om.Problem()

comp = om.VectorMagnitudeComp(vec_size=n, length=3,
                              in_name='r', mag_name='r_mag', units='km')

comp.add_magnitude(vec_size=n, length=3,
                   in_name='b', mag_name='b_mag', units='ft')

p.model.add_subsystem(name='vec_mag_comp', subsys=comp,
                      promotes_inputs=['r', 'b'])

p.setup()

p.set_val('r', 1.0 + np.random.rand(n, 3))
p.set_val('b', 1.0 + np.random.rand(n, 3))

p.run_model()

# Verify the results against numpy.dot in a for loop.
expected_r = []
expected_b = []
for i in range(n):
    a_i = p.get_val('r')[i, :]
    expected_r.append(np.sqrt(np.dot(a_i, a_i)))

    actual_i = p.get_val('vec_mag_comp.r_mag')[i]
    rel_error = np.abs(expected_r[i] - actual_i)/actual_i
    assert rel_error < 1e-9, f"Relative error: {rel_error}"

    b_i = p.get_val('b')[i, :]
    expected_b.append(np.sqrt(np.dot(b_i, b_i)))

    actual_i = p.get_val('vec_mag_comp.b_mag')[i]
    rel_error = np.abs(expected_b[i] - actual_i)/actual_i
    assert rel_error < 1e-9, f"Relative error: {rel_error}"

print(p.get_val('vec_mag_comp.r_mag'))

In [ ]:
print(p.get_val('vec_mag_comp.b_mag'))

In [ ]:
assert_near_equal(p.get_val('vec_mag_comp.r_mag'), np.array(expected_r))
assert_near_equal(p.get_val('vec_mag_comp.b_mag'), np.array(expected_b))